_Backtesting.py_ 快速入门用户指南
=======================
Backtesting.py快速入门用户指南
本教程展示了 backtesting.py 的一些功能，这是一个用于回溯测试交易策略的Python框架。

Backtesting.py是一个小巧轻量级，快速的回溯测试框架，使用最先进的Python结构和程序（Python 3.6 +，Pandas，NumPy，Bokeh）。它有一个非常小而简单的API，易于记忆并快速形成有意义的结果。该库并不真正支持依赖于套利或多资产投资组合再平衡的选股或交易策略;相反，它一次可与单个可交易资产一起使用，最适合优化仓位进入和退出信号策略，决定技术指标的值，它也是一个多功能的交互式交易可视化和统计工具。

## 数据
您自带数据。回溯测试摄取_all种OHLC data_（股票，外汇，期货，加密货币等）作为熊猫。包含列、、、 和（可选）的数据帧。这样的数据是广泛可获得的（参见：pandas-datareader， Quandl， findatapy）。除此之外，您的数据框还可以具有其他列，这些列可以在您的策略中以类似的方式访问。'Open''High''Low''Close''Volume'

理想情况下，DataFrame应该使用日期时间索引（将其转换为pd.to_datetime（）），否则可以使用简单的范围索引。


## Data
_You bring your own data._ Backtesting ingests _all kinds of 
[OHLC](https://en.wikipedia.org/wiki/Open-high-low-close_chart)
data_ (stocks, forex, futures, crypto, ...) as a
[pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/10min.html)
with columns `'Open'`, `'High'`, `'Low'`, `'Close'` and (optionally) `'Volume'`. Such data is widely obtainable (see: 
[pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/),
[Quandl](https://www.quandl.com/tools/python),
[findatapy](https://github.com/cuemacro/findatapy)).
Besides these, your data frames can have _additional columns_ which are accessible in your strategies in a similar manner.

DataFrame should ideally be indexed with a _datetime index_ (convert it with [`pd.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)), otherwise a simple range index will do.

In [26]:
# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG

GOOG.tail()

,Open,High,Low,Close,Volume
2013-02-25,802.3,808.41,790.49,790.77,2303900
2013-02-26,795.0,795.95,784.40,790.13,2202500
2013-02-27,794.8,804.75,791.11,799.78,2026100
2013-02-28,801.1,806.99,801.03,801.20,2265800
2013-03-01,797.8,807.14,796.15,806.19,2175400


## 策略
让我们创建第一个策略来回溯测试这些Google数据，这是一个简单的移动平均线（MA）交叉策略。

Backtesting.py没有发布自己的一套技术分析指标。偏爱 TA 的用户可能应该参考经过验证的指标库中的函数，例如TA-Lib或Tulipy，但对于此示例，我们可以自己定义一个简单的帮助器移动平均函数：

In [27]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

新的策略需要扩展Strategy类并重写其两个抽象方法：init（）和next（）。

在运行策略之前调用方法。在其中，人们理想地以高效，矢量化的方式预先计算策略所依赖的任何指标和信号。init()

然后，Backtest实例以迭代方式调用方法，为每个数据点（数据帧行）调用一次，模拟每个新的完整蜡烛条的增量可用性。next()

请注意，backtesting.py不能在烛台内做出决定/交易 - 任何新订单都会在下一根蜡烛的开盘时执行（如果trade_on_close=True，则当前蜡烛的收盘价）。如果您发现自己希望在烛台内进行交易（例如日间交易），则需要从更细粒度（例如每小时）的数据开始。


In [28]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

在 和 中，模拟策略的数据可用作实例变量self.data。init()next()

在 中，我们通过将指标包装在 self 中来间接声明和计算指标。I（）.包装器被传递给一个函数（我们的函数）以及任何用于调用它的参数（我们的关闭值和 MA 滞后）。以这种方式包装的指标将自动绘制，并且将智能地推断其图例字符串。init()SMA

在 中，我们只需检查较快的移动平均线是否刚好越过较慢的移动平均线。如果确实如此并且向上，我们关闭可能的空头头寸并做多;如果确实如此，并且向下，我们关闭未平仓多头头寸并做空。请注意，我们不会调整订单大小，因此Backtesting.py假设最大可能位置。我们使用backtesting.lib.crossover（）函数，而不是编写更晦涩和令人困惑的条件，例如：next()

In [29]:
%%script echo

    def next(self):
        if (self.sma1[-2] < self.sma2[-2] and
                self.sma1[-1] > self.sma2[-1]):
            self.position.close()
            self.buy()

        elif (self.sma1[-2] > self.sma2[-2] and    # Ugh!
              self.sma1[-1] < self.sma2[-1]):
            self.position.close()
            self.sell()

Couldn't find program: 'echo'


在 中，整个点系列都可用，而在next（）中，self.data的长度和所有声明的指标在每次调用时都会进行调整，以便（例如 或 ） 始终包含最新的值、以前的值等（升序排序的 1D 数组的普通 Python 索引）。init()next()array[-1]self.data.Close[-1]self.sma1[-1]array[-2]

注意：出于性能原因，任何包装的指标（例如）都是NumPy数组。如果您更喜欢 pandas 系列或 DataFrame 对象，请分别使用或访问器。您也可以手动构造序列，例如.self.dataself.Iself.sma1Strategy.data.<column>.sStrategy.data.dfpd.Series(self.data.Close, index=self.data.index)

如果我们使用 . self.position.close()Backtest(..., exclusive_orders=True)



## 回溯测试
让我们看看我们的策略在历史 Google 数据上的表现如何。回溯测试实例使用 OHLC 数据和策略类进行初始化（有关其他选项，请参阅 API 参考），我们从 10，000 个现金单位开始，并将经纪商的佣金设置为实际的 0.2%。

In [30]:
from backtesting import Backtest

bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   97.067039
Equity Final [$]                  68221.96986
Equity Peak [$]                   68991.21986
Return [%]                         582.219699
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   25.266427
Volatility (Ann.) [%]               38.383008
Sharpe Ratio                         0.658271
Sortino Ratio                        1.288779
Calmar Ratio                         0.763748
Max. Drawdown [%]                  -33.082172
Avg. Drawdown [%]                   -5.581506
Max. Drawdown Duration      688 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   94
Win Rate [%]                        54.255319
Best Trade [%]                       57.11931
Worst Trade [%]                    -16.629898
Avg. Trade [%]                    

Backtest.run（）方法返回一个 pandas 一系列与我们的策略相关的模拟结果和统计信息。我们看到，这种简单的策略在9年的时间里获得了近600%的回报，最大回撤率为33%，最长的回撤期跨越了近两年......

Backtest.plot（）方法以更直观的形式提供相同的见解。

In [31]:
bt.plot()

Row(id='3952', ...)

## 优化
我们将两个滞后参数（和）硬编码到上面的策略中。但是，该策略在15-30或其他交叉时可能会更好。我们通过使它们成为类变量来声明参数是可优化的。n1n2

我们通过调用Backtest.optimize（）方法来优化这两个参数，每个参数都有一个关键字参数，指向其可能要测试的值池。参数分别针对介于 5 和 30 之间的值和 10 和 70 之间的值进行测试。两个参数的某些值组合无效，即 不应大于或等于 。我们使用临时约束函数限制可接受的参数组合，该函数接收参数并在小于 时返回（即可接受的）。此外，我们搜索这样的参数组合，该组合在观察到的时间段内最大化回报。相反，我们可以选择优化返回的序列中的任何其他密钥。n1n2n1n2Truen1n2stats

In [32]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

  0%|          | 0/17 [00:00<?, ?it/s]

Wall time: 4.67 s


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   99.068901
Equity Final [$]                 103949.42612
Equity Peak [$]                  108327.71798
Return [%]                         939.494261
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   31.610936
Volatility (Ann.) [%]               44.739816
Sharpe Ratio                          0.70655
Sortino Ratio                        1.490961
Calmar Ratio                         0.718505
Max. Drawdown [%]                  -43.995445
Avg. Drawdown [%]                   -6.138853
Max. Drawdown Duration      690 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                  153
Win Rate [%]                        51.633987
Best Trade [%]                      61.562908
Worst Trade [%]                    -19.778312
Avg. Trade [%]                    

我们可以查看策略实例及其最佳参数值（10 和 15）。stats['_strategy']

In [33]:
stats._strategy

<Strategy SmaCross(n1=10,n2=15)>

In [34]:
bt.plot(plot_volume=False, plot_pl=False)

Row(id='4891', ...)

策略优化设法将其样本数据上的初始性能提高了近50%，甚至击败了简单的买入和持有。但是，在现实生活中的优化中，请采取措施避免过度拟合。

## 交易数据
除了上面显示的Backtest.run（）返回的回溯测试统计数据外，您还可以通过检查结果序列中的最后几个内部键来查看单个交易回报以及不断变化的净值曲线和回撤。

In [35]:
stats.tail()

Expectancy [%]                                              1.97988
SQN                                                        1.604158
_strategy                                     SmaCross(n1=10,n2=15)
_equity_curve                       Equity  DrawdownPct Drawdown...
_trades                Size  EntryBar  ExitBar  EntryPrice  Exit...
dtype: object

这些列应不言自明。

In [36]:
stats['_equity_curve']  # Contains equity/drawdown curves. DrawdownDuration is only defined at ends of DD periods.

,Equity,DrawdownPct,DrawdownDuration
2004-08-19,10000.00000,0.000000,NaT
2004-08-20,10000.00000,0.000000,NaT
2004-08-23,10000.00000,0.000000,NaT
2004-08-24,10000.00000,0.000000,NaT
2004-08-25,10000.00000,0.000000,NaT
...,...,...,...
2013-02-25,103035.52612,0.048854,NaT
2013-02-26,102952.32612,0.049622,NaT
2013-02-27,104206.82612,0.038041,NaT
2013-02-28,104391.42612,0.036337,NaT


In [37]:
stats['_trades']  # Contains individual trade data

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,87,20,60,114.64884,185.23,6140.56092,0.615629,2004-09-17,2004-11-12,56 days
1,-87,60,69,184.85954,175.80,788.17998,0.049008,2004-11-12,2004-11-26,14 days
2,96,69,71,176.15160,180.71,437.60640,0.025878,2004-11-26,2004-11-30,4 days
3,-96,71,75,180.34858,179.13,116.98368,0.006757,2004-11-30,2004-12-06,6 days
4,97,75,82,179.48826,177.99,-145.33122,-0.008347,2004-12-06,2004-12-15,9 days
...,...,...,...,...,...,...,...,...,...,...
148,139,2085,2111,689.15556,735.54,6447.43716,0.067306,2012-11-29,2013-01-08,40 days
149,-139,2111,2113,734.06892,742.83,-1217.79012,-0.011935,2013-01-08,2013-01-10,2 days
150,136,2113,2121,744.31566,735.99,-1132.28976,-0.011186,2013-01-10,2013-01-23,13 days
151,-136,2121,2127,734.51802,750.51,-2174.90928,-0.021772,2013-01-23,2013-01-31,8 days


Learn more by exploring further
[examples](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#tutorials)
or find more framework options in the
[full API reference](https://kernc.github.io/backtesting.py/doc/backtesting/index.html#header-submodules).
通过探索更多示例了解更多信息，或在完整的 API 参考中找到更多框架选项。